In [3]:
# =============================================================================
# DISH DEMAND FORECASTER – MODEL COMPARISON NOTEBOOK
# Author: Your Name
# Date: 2025-10-27
# Goal: Compare R² scores of Random Forest, XGBoost, and Feedforward Neural Network
# =============================================================================

# --------------------------------------------------------------
# 1. IMPORTS & CONFIG
# --------------------------------------------------------------
import os
import re
import joblib
import warnings
from collections import Counter
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.neural_network import MLPRegressor

warnings.filterwarnings("ignore")

# --- CONFIG ---
DATA_PATH = "../data/data.csv"
TOP_K = 100
LAGS = [1, 2, 3, 6, 12, 24]
WINDOWS = [3, 6, 12, 24]
USE_SCALER = True
TRAIN_FRAC = 0.8
RANDOM_STATE = 42
MODEL_DIR = "models"
CV_FOLDS = 5
os.makedirs(MODEL_DIR, exist_ok=True)

print("Configuration loaded.\n")

# --------------------------------------------------------------
# 2. PARSING UTILITIES
# --------------------------------------------------------------
def parse_order_items(order_str: str) -> List[Tuple[str, int]]:
    """Parse '2 x Pizza, 1 x Coke' → [('Pizza', 2), ('Coke', 1)]"""
    if pd.isna(order_str):
        return []
    return [(name.strip(), int(qty)) for qty, name in re.findall(r"(\d+)\s*[xX]\s*([^,]+)", order_str)]

def expand_items(df: pd.DataFrame) -> pd.DataFrame:
    """Expand 'Items in order' into a list of (item, qty) tuples."""
    df["expanded_items"] = df["Items in order"].fillna("").apply(parse_order_items)
    return df

print("Parsing utilities ready.\n")

# --------------------------------------------------------------
# 3. LOAD & PREPROCESS
# --------------------------------------------------------------
def load_and_prepare_data(path: str) -> Tuple[pd.DataFrame, Dict[str, List[str]]]:
    """Load and preprocess raw order data, returning DataFrame and restaurant menus."""
    df = pd.read_csv(path)
    print(f"Raw data: {df.shape[0]:,} rows, {df.shape[1]} columns")

    dt_col = next((col for col in df.columns
                   if "order" in col.lower() and ("placed" in col.lower() or "date" in col.lower())), None)
    if not dt_col:
        raise ValueError("No order datetime column found.")
    df["order_datetime"] = pd.to_datetime(df[dt_col], errors="coerce")
    df = df.dropna(subset=["order_datetime"]).copy()
    df["order_hour"] = df["order_datetime"].dt.floor("h")

    rest_col = "Restaurant name"
    if rest_col not in df.columns:
        raise ValueError(f"'{rest_col}' column not found.")
    df[rest_col] = df[rest_col].str.strip()
    unique_restaurants = df[rest_col].unique()
    print(f"Found {len(unique_restaurants)} restaurants: {list(unique_restaurants)}")
    df = pd.get_dummies(df, columns=[rest_col], prefix="rest", dtype=int)

    if "Items in order" not in df.columns:
        raise ValueError("'Items in order' column not found.")
    df = expand_items(df)

    rest_menus = {}
    rest_cols = [col for col in df.columns if col.startswith("rest_")]
    for rest_col in rest_cols:
        rest_name = rest_col[5:]  # Remove 'rest_' prefix
        rest_orders = df[df[rest_col] == 1]["expanded_items"]
        menu = set()
        for items in rest_orders:
            menu.update(item for item, _ in items)
        rest_menus[rest_name] = sorted(list(menu))
    print(f"Restaurant menus created: { {k: len(v) for k, v in rest_menus.items()} }")
    print("Swaad's menu:", rest_menus.get("Swaad", []))

    print(f"Pre-processed: {len(df):,} orders")
    return df, rest_menus

df, rest_menus = load_and_prepare_data(DATA_PATH)

# --------------------------------------------------------------
# 4. BUILD HOURLY AGGREGATED TABLE
# --------------------------------------------------------------
def build_hourly_table(df: pd.DataFrame, top_k: int) -> Tuple[pd.DataFrame, List[str]]:
    """Aggregate orders to hourly level with top-K dishes."""
    all_items = Counter()
    for items in df["expanded_items"]:
        all_items.update({item: qty for item, qty in items})
    top_dishes = [name for name, _ in all_items.most_common(top_k)]
    print(f"Top-{len(top_dishes)} dishes selected (sample: {top_dishes[:5]})")

    hour_idx = pd.date_range(
        start=df["order_hour"].min().floor("D"),
        end=df["order_hour"].max().ceil("D"),
        freq="h"
    )
    agg = pd.DataFrame(index=hour_idx)
    agg.index.name = "order_hour"

    agg["hour_of_day"] = agg.index.hour
    agg["day_of_week"] = agg.index.dayofweek
    agg["is_weekend"] = agg.index.dayofweek.isin([5, 6]).astype(int)

    for dish in top_dishes:
        agg[f"dish__{dish}"] = 0
    agg["total_orders"] = 0

    rest_cols = [col for col in df.columns if col.startswith("rest_")]
    for col in rest_cols:
        agg[col] = 0

    for hour, group in df.groupby("order_hour"):
        if hour not in agg.index:
            continue
        agg.loc[hour, "total_orders"] = len(group)
        for col in rest_cols:
            agg.loc[hour, col] = 1 if group[col].sum() > 0 else 0
        hour_counts = Counter()
        for items in group["expanded_items"]:
            hour_counts.update({item: qty for item, qty in items})
        for dish in top_dishes:
            agg.loc[hour, f"dish__{dish}"] = hour_counts.get(dish, 0)

    agg = agg[agg["total_orders"] > 0].copy()
    print(f"Aggregated {len(agg)} active hours.")
    return agg, top_dishes

agg, top_dishes = build_hourly_table(df, TOP_K)

# --------------------------------------------------------------
# 5. FEATURE ENGINEERING (lags + rolling)
# --------------------------------------------------------------
def add_temporal_features(agg: pd.DataFrame, lags: List[int], windows: List[int]) -> pd.DataFrame:
    """Add lag and rolling window features."""
    df = agg.copy()

    for lag in lags:
        df[f"total_orders_lag_{lag}"] = df["total_orders"].shift(lag)
        for dish in top_dishes:
            df[f"dish__{dish}_lag_{lag}"] = df[f"dish__{dish}"].shift(lag)

    for w in windows:
        df[f"total_orders_rollmean_{w}"] = df["total_orders"].rolling(w, min_periods=1).mean()
        for dish in top_dishes:
            df[f"dish__{dish}_rollmean_{w}"] = df[f"dish__{dish}"].rolling(w, min_periods=1).mean()

    df = df.dropna().reset_index()  # Keep order_hour as column
    print(f"Feature engineering → {df.shape[0]} rows, {df.shape[1]} cols")
    return df

agg_with_hour = add_temporal_features(agg, LAGS, WINDOWS)

# --------------------------------------------------------------
# 6. TRAIN / TEST SPLIT (time-based)
# --------------------------------------------------------------
def ensure_restaurant_coverage(train_agg: pd.DataFrame, test_agg: pd.DataFrame,
                              rest_cols: List[str]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Ensure all restaurants appear in training data."""
    train_rest = set([col for col in rest_cols if train_agg[col].sum() > 0])
    test_rest = set([col for col in rest_cols if test_agg[col].sum() > 0])
    missing = test_rest - train_rest
    if missing:
        print(f"Warning: {len(missing)} restaurants in test but not in train: {missing}")
        for col in missing:
            rest_data = test_agg[test_agg[col] == 1].head(1)
            if not rest_data.empty:
                train_agg = pd.concat([train_agg, rest_data])
                test_agg = test_agg.drop(rest_data.index)
        print("Adjusted train/test split to include all restaurants.")
    return train_agg, test_agg

split_time = agg_with_hour["order_hour"].quantile(TRAIN_FRAC)
train_agg = agg_with_hour[agg_with_hour["order_hour"] < split_time].copy()
test_agg = agg_with_hour[agg_with_hour["order_hour"] >= split_time].copy()
rest_cols = [col for col in agg_with_hour.columns if col.startswith("rest_")]
train_agg, test_agg = ensure_restaurant_coverage(train_agg, test_agg, rest_cols)
full_agg = agg_with_hour.set_index("order_hour")

print(f"Train hours: {len(train_agg)} | Test hours: {len(test_agg)}")

# --------------------------------------------------------------
# 7. FEATURE / TARGET LISTS
# --------------------------------------------------------------
feature_cols = [col for col in train_agg.columns if not col.startswith("dish__") and col != "order_hour"]
rest_cols = [col for col in feature_cols if col.startswith("rest_")]
target_cols = [f"dish__{dish}" for dish in top_dishes]

print(f"Features: {len(feature_cols)} (restaurants: {len(rest_cols)})")
print(f"Targets: {len(target_cols)}")

# --------------------------------------------------------------
# 8. MODEL COMPARISON
# --------------------------------------------------------------
def evaluate_model(model, model_name, X_train, Y_train, X_test, Y_test):
    """Train model and return test R² score."""
    multi_model = MultiOutputRegressor(model)
    multi_model.fit(X_train, Y_train)
    preds = multi_model.predict(X_test)
    r2 = r2_score(Y_test, preds, multioutput='uniform_average')
    print(f"{model_name} Test R²: {r2:.3f}")
    return r2

# Prepare data
if USE_SCALER:
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(train_agg[feature_cols]), columns=feature_cols)
    X_test_scaled = pd.DataFrame(scaler.transform(test_agg[feature_cols]), columns=feature_cols)
else:
    X_train_scaled = train_agg[feature_cols].copy()
    X_test_scaled = test_agg[feature_cols].copy()

Y_train = train_agg[target_cols].values
Y_test = test_agg[target_cols].values

# Models
models = {
    "Random Forest": RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1),
    "XGBoost": xgb.XGBRegressor(random_state=RANDOM_STATE, n_jobs=-1),
    "Feedforward NN": MLPRegressor(random_state=RANDOM_STATE, max_iter=500)
}

# Evaluate all models
results = {}
for name, model in models.items():
    r2 = evaluate_model(model, name, X_train_scaled.values, Y_train, X_test_scaled.values, Y_test)
    results[name] = r2

# Print summary
print("\nModel Comparison Summary:")
for name, r2 in results.items():
    print(f"{name}: R² = {r2:.3f}")
best_model_name = max(results, key=results.get)
print(f"Best model: {best_model_name} with R² = {results[best_model_name]:.3f}")

# --------------------------------------------------------------
# 9. SAVE BEST MODEL
# --------------------------------------------------------------
best_model = MultiOutputRegressor(models[best_model_name])
best_model.fit(X_train_scaled.values, Y_train)
joblib.dump({
    "model": best_model,
    "scaler": scaler if USE_SCALER else None,
    "feature_cols": feature_cols,
    "target_cols": target_cols,
    "full_agg": full_agg
}, os.path.join(MODEL_DIR, "best_model_comparison.pkl"))

print(f"\nBest model saved → {MODEL_DIR}/best_model_comparison.pkl")

Configuration loaded.

Parsing utilities ready.

Raw data: 21,321 rows, 29 columns
Found 6 restaurants: ['Swaad', 'Aura Pizzas', 'Dilli Burger Adda', 'Tandoori Junction', 'The Chicken Junction', 'Masala Junction']
Restaurant menus created: {'Aura Pizzas': 76, 'Dilli Burger Adda': 64, 'Masala Junction': 13, 'Swaad': 94, 'Tandoori Junction': 37, 'The Chicken Junction': 21}
Swaad's menu: ['AAC Fried Chicken Burger', 'AAC Fried Paneer Burger', 'AAC Grilled Chicken Burger', 'AAC Grilled Chicken Burger .', 'AAC Grilled Paneer Burger', 'AAC Saucy Fries', 'AAC Signature Crisper Fries', 'AAC Signature Fries', 'AAC Signature Krispers', 'AAC Special Slaw', 'Angara Aloo Tuk Tuki', 'Angara Grilled Paneer (8 pcs)', 'Angara Rice', 'Angara Sauce', 'Animal Fries', 'Animal Fries .', 'Bone in Angara Grilled Chicken', 'Bone in Angara Grilled Chicken .', 'Bone in Jamaican Grilled Chicken', 'Bone in Jamaican Grilled Chicken .', 'Bone in Jamaican Grilled Chicken Quarter + Angara Rice', 'Bone in Kabuli Grille

In [ ]:
# =============================================================================
# DISH DEMAND FORECASTER – RANDOM FOREST HYPERPARAMETER TUNING
# Author: Your Name
# Date: 2025-10-27
# Goal: Optimize Random Forest for predicting hourly demand for top-K dishes
# =============================================================================

# --------------------------------------------------------------
# 1. IMPORTS & CONFIG
# --------------------------------------------------------------
import os
import re
import joblib
import warnings
from collections import Counter
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

warnings.filterwarnings("ignore")

# --- CONFIG ---
DATA_PATH = "../data/data.csv"
TOP_K = 50
LAGS = [1, 2, 3, 6, 12, 24]
WINDOWS = [3, 6, 12, 24]
USE_SCALER = True
TRAIN_FRAC = 0.8
RANDOM_STATE = 42
MODEL_DIR = "models"
CV_FOLDS = 5
os.makedirs(MODEL_DIR, exist_ok=True)

print("Configuration loaded.\n")

# --------------------------------------------------------------
# 2. PARSING UTILITIES
# --------------------------------------------------------------
def parse_order_items(order_str: str) -> List[Tuple[str, int]]:
    """Parse '2 x Pizza, 1 x Coke' → [('Pizza', 2), ('Coke', 1)]"""
    if pd.isna(order_str):
        return []
    return [(name.strip(), int(qty)) for qty, name in re.findall(r"(\d+)\s*[xX]\s*([^,]+)", order_str)]

def expand_items(df: pd.DataFrame) -> pd.DataFrame:
    """Expand 'Items in order' into a list of (item, qty) tuples."""
    df["expanded_items"] = df["Items in order"].fillna("").apply(parse_order_items)
    return df

print("Parsing utilities ready.\n")

# --------------------------------------------------------------
# 3. LOAD & PREPROCESS
# --------------------------------------------------------------
def load_and_prepare_data(path: str) -> Tuple[pd.DataFrame, Dict[str, List[str]]]:
    """Load and preprocess raw order data, returning DataFrame and restaurant menus."""
    df = pd.read_csv(path)
    print(f"Raw data: {df.shape[0]:,} rows, {df.shape[1]} columns")

    dt_col = next((col for col in df.columns
                   if "order" in col.lower() and ("placed" in col.lower() or "date" in col.lower())), None)
    if not dt_col:
        raise ValueError("No order datetime column found.")
    df["order_datetime"] = pd.to_datetime(df[dt_col], errors="coerce")
    df = df.dropna(subset=["order_datetime"]).copy()
    df["order_hour"] = df["order_datetime"].dt.floor("h")

    rest_col = "Restaurant name"
    if rest_col not in df.columns:
        raise ValueError(f"'{rest_col}' column not found.")
    df[rest_col] = df[rest_col].str.strip()
    unique_restaurants = df[rest_col].unique()
    print(f"Found {len(unique_restaurants)} restaurants: {list(unique_restaurants)}")
    df = pd.get_dummies(df, columns=[rest_col], prefix="rest", dtype=int)

    if "Items in order" not in df.columns:
        raise ValueError("'Items in order' column not found.")
    df = expand_items(df)

    rest_menus = {}
    rest_cols = [col for col in df.columns if col.startswith("rest_")]
    for rest_col in rest_cols:
        rest_name = rest_col[5:]  # Remove 'rest_' prefix
        rest_orders = df[df[rest_col] == 1]["expanded_items"]
        menu = set()
        for items in rest_orders:
            menu.update(item for item, _ in items)
        rest_menus[rest_name] = sorted(list(menu))
    print(f"Restaurant menus created: { {k: len(v) for k, v in rest_menus.items()} }")
    print("Swaad's menu:", rest_menus.get("Swaad", []))

    print(f"Pre-processed: {len(df):,} orders")
    return df, rest_menus

df, rest_menus = load_and_prepare_data(DATA_PATH)

# --------------------------------------------------------------
# 4. BUILD HOURLY AGGREGATED TABLE
# --------------------------------------------------------------
def build_hourly_table(df: pd.DataFrame, top_k: int) -> Tuple[pd.DataFrame, List[str]]:
    """Aggregate orders to hourly level with top-K dishes."""
    all_items = Counter()
    for items in df["expanded_items"]:
        all_items.update({item: qty for item, qty in items})
    top_dishes = [name for name, _ in all_items.most_common(top_k)]
    print(f"Top-{len(top_dishes)} dishes selected (sample: {top_dishes[:5]})")

    hour_idx = pd.date_range(
        start=df["order_hour"].min().floor("D"),
        end=df["order_hour"].max().ceil("D"),
        freq="h"
    )
    agg = pd.DataFrame(index=hour_idx)
    agg.index.name = "order_hour"

    agg["hour_of_day"] = agg.index.hour
    agg["day_of_week"] = agg.index.dayofweek
    agg["is_weekend"] = agg.index.dayofweek.isin([5, 6]).astype(int)

    for dish in top_dishes:
        agg[f"dish__{dish}"] = 0
    agg["total_orders"] = 0

    rest_cols = [col for col in df.columns if col.startswith("rest_")]
    for col in rest_cols:
        agg[col] = 0

    for hour, group in df.groupby("order_hour"):
        if hour not in agg.index:
            continue
        agg.loc[hour, "total_orders"] = len(group)
        for col in rest_cols:
            agg.loc[hour, col] = 1 if group[col].sum() > 0 else 0
        hour_counts = Counter()
        for items in group["expanded_items"]:
            hour_counts.update({item: qty for item, qty in items})
        for dish in top_dishes:
            agg.loc[hour, f"dish__{dish}"] = hour_counts.get(dish, 0)

    agg = agg[agg["total_orders"] > 0].copy()
    print(f"Aggregated {len(agg)} active hours.")
    return agg, top_dishes

agg, top_dishes = build_hourly_table(df, TOP_K)

# --------------------------------------------------------------
# 5. FEATURE ENGINEERING (lags + rolling)
# --------------------------------------------------------------
def add_temporal_features(agg: pd.DataFrame, lags: List[int], windows: List[int]) -> pd.DataFrame:
    """Add lag and rolling window features."""
    df = agg.copy()

    for lag in lags:
        df[f"total_orders_lag_{lag}"] = df["total_orders"].shift(lag)
        for dish in top_dishes:
            df[f"dish__{dish}_lag_{lag}"] = df[f"dish__{dish}"].shift(lag)

    for w in windows:
        df[f"total_orders_rollmean_{w}"] = df["total_orders"].rolling(w, min_periods=1).mean()
        for dish in top_dishes:
            df[f"dish__{dish}_rollmean_{w}"] = df[f"dish__{dish}"].rolling(w, min_periods=1).mean()

    df = df.dropna().reset_index()  # Keep order_hour as column
    print(f"Feature engineering → {df.shape[0]} rows, {df.shape[1]} cols")
    return df

agg_with_hour = add_temporal_features(agg, LAGS, WINDOWS)

# --------------------------------------------------------------
# 6. TRAIN / TEST SPLIT (time-based)
# --------------------------------------------------------------
def ensure_restaurant_coverage(train_agg: pd.DataFrame, test_agg: pd.DataFrame,
                              rest_cols: List[str]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Ensure all restaurants appear in training data."""
    train_rest = set([col for col in rest_cols if train_agg[col].sum() > 0])
    test_rest = set([col for col in rest_cols if test_agg[col].sum() > 0])
    missing = test_rest - train_rest
    if missing:
        print(f"Warning: {len(missing)} restaurants in test but not in train: {missing}")
        for col in missing:
            rest_data = test_agg[test_agg[col] == 1].head(1)
            if not rest_data.empty:
                train_agg = pd.concat([train_agg, rest_data])
                test_agg = test_agg.drop(rest_data.index)
        print("Adjusted train/test split to include all restaurants.")
    return train_agg, test_agg

split_time = agg_with_hour["order_hour"].quantile(TRAIN_FRAC)
train_agg = agg_with_hour[agg_with_hour["order_hour"] < split_time].copy()
test_agg = agg_with_hour[agg_with_hour["order_hour"] >= split_time].copy()
rest_cols = [col for col in agg_with_hour.columns if col.startswith("rest_")]
train_agg, test_agg = ensure_restaurant_coverage(train_agg, test_agg, rest_cols)
full_agg = agg_with_hour.set_index("order_hour")

print(f"Train hours: {len(train_agg)} | Test hours: {len(test_agg)}")

# --------------------------------------------------------------
# 7. FEATURE / TARGET LISTS
# --------------------------------------------------------------
feature_cols = [col for col in train_agg.columns if not col.startswith("dish__") and col != "order_hour"]
rest_cols = [col for col in feature_cols if col.startswith("rest_")]
target_cols = [f"dish__{dish}" for dish in top_dishes]

print(f"Features: {len(feature_cols)} (restaurants: {len(rest_cols)})")
print(f"Targets: {len(target_cols)}")

# --------------------------------------------------------------
# 8. HYPERPARAMETER TUNING FOR RANDOM FOREST
# --------------------------------------------------------------
def tune_random_forest(X: np.ndarray, Y: np.ndarray, cv_splits):
    """Tune Random Forest hyperparameters using GridSearchCV for multi-output."""
    base_estimator = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1)
    model = MultiOutputRegressor(base_estimator)

    param_grid = {
        'estimator__n_estimators': [100, 200, 300],
        'estimator__max_depth': [10, 20, 30, None],
        'estimator__min_samples_split': [2, 5, 10],
        'estimator__min_samples_leaf': [1, 2, 4]
    }

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='r2',
        cv=cv_splits,
        n_jobs=-1,
        verbose=1
    )

    print("Starting hyperparameter tuning for Random Forest...")
    grid_search.fit(X, Y)
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV R²: {grid_search.best_score_:.3f}")
    return grid_search.best_estimator_

# Prepare data
if USE_SCALER:
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(train_agg[feature_cols]), columns=feature_cols)
    X_test_scaled = pd.DataFrame(scaler.transform(test_agg[feature_cols]), columns=feature_cols)
else:
    X_train_scaled = train_agg[feature_cols].copy()
    X_test_scaled = test_agg[feature_cols].copy()

Y_train = train_agg[target_cols].values
Y_test = test_agg[target_cols].values

tscv = TimeSeriesSplit(n_splits=CV_FOLDS)
best_rf_model = tune_random_forest(X_train_scaled.values, Y_train, tscv)

# --------------------------------------------------------------
# 9. EVALUATE ON TEST SET
# --------------------------------------------------------------
preds = best_rf_model.predict(X_test_scaled.values)
test_r2 = r2_score(Y_test, preds, multioutput='uniform_average')
print(f"\nTest R² with best Random Forest: {test_r2:.3f}")

# --------------------------------------------------------------
# 10. SAVE MODEL
# --------------------------------------------------------------
joblib.dump({
    "model": best_rf_model,
    "scaler": scaler if USE_SCALER else None,
    "feature_cols": feature_cols,
    "target_cols": target_cols,
    "full_agg": full_agg
}, os.path.join(MODEL_DIR, "best_rf_tuned.pkl"))

print(f"\nModel saved → {MODEL_DIR}/best_rf_tuned.pkl")

Configuration loaded.

Parsing utilities ready.

Raw data: 21,321 rows, 29 columns
Found 6 restaurants: ['Swaad', 'Aura Pizzas', 'Dilli Burger Adda', 'Tandoori Junction', 'The Chicken Junction', 'Masala Junction']
Restaurant menus created: {'Aura Pizzas': 76, 'Dilli Burger Adda': 64, 'Masala Junction': 13, 'Swaad': 94, 'Tandoori Junction': 37, 'The Chicken Junction': 21}
Swaad's menu: ['AAC Fried Chicken Burger', 'AAC Fried Paneer Burger', 'AAC Grilled Chicken Burger', 'AAC Grilled Chicken Burger .', 'AAC Grilled Paneer Burger', 'AAC Saucy Fries', 'AAC Signature Crisper Fries', 'AAC Signature Fries', 'AAC Signature Krispers', 'AAC Special Slaw', 'Angara Aloo Tuk Tuki', 'Angara Grilled Paneer (8 pcs)', 'Angara Rice', 'Angara Sauce', 'Animal Fries', 'Animal Fries .', 'Bone in Angara Grilled Chicken', 'Bone in Angara Grilled Chicken .', 'Bone in Jamaican Grilled Chicken', 'Bone in Jamaican Grilled Chicken .', 'Bone in Jamaican Grilled Chicken Quarter + Angara Rice', 'Bone in Kabuli Grille